<a href="https://colab.research.google.com/github/hsnbnaa/deep-learning-is-easy/blob/main/Hands-on%20Machine%20Learning%20with%20Scikit-Learn%2C%20Keras%20%26%20TensorFlow/Chapter%2019/19_training_and_deploying_at_scale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow import keras
import os

# Contoh sederhana model untuk MNIST
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Dummy training (gunakan data MNIST asli jika diperlukan)
(x_train, y_train), _ = keras.datasets.mnist.load_data()
x_train = x_train / 255.0
model.fit(x_train, y_train, epochs=1)

# Menyimpan model dalam format untuk TensorFlow Serving
model_version = "1"
model_name = "my_mnist_model"
model_path = os.path.join("models", model_name, model_version)
os.makedirs(model_path, exist_ok=True)

model.export(model_path)
print(f"Model exported to {model_path}")


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8783 - loss: 0.4308
Saved artifact at 'models/my_mnist_model/1'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='keras_tensor_4')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  134992478588368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134992478597008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134992478596816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134992478596048: TensorSpec(shape=(), dtype=tf.resource, name=None)
Model exported to models/my_mnist_model/1


In [3]:
import json
import requests
import numpy as np
from tensorflow import keras

# Contoh memuat data MNIST untuk prediksi
(_, _), (X_test, y_test) = keras.datasets.mnist.load_data()
X_test = X_test / 255.0  # Normalisasi sama seperti saat training
X_new = X_test[:3]       # Ambil 3 gambar untuk prediksi
X_new = X_new.tolist()   # Konversi ke format JSON-friendly

# Membuat request JSON
request_json = json.dumps({
    "signature_name": "serving_default",  # default signature
    "instances": X_new
})

# URL endpoint model (pastikan TensorFlow Serving sudah jalan)
server_url = "http://localhost:8501/v1/models/my_mnist_model:predict"

# Kirim permintaan POST ke TF Serving
response = requests.post(server_url, data=request_json, headers={"Content-Type": "application/json"})

# Cek jika respons berhasil
if response.status_code == 200:
    predictions = response.json()["predictions"]
    print("Predictions:")
    for i, pred in enumerate(predictions):
        predicted_class = np.argmax(pred)
        print(f"Sample {i}: Predicted class = {predicted_class}")
else:
    print("Error:", response.status_code)
    print(response.text)


ConnectionError: HTTPConnectionPool(host='localhost', port=8501): Max retries exceeded with url: /v1/models/my_mnist_model:predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7ac704de9b90>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
import tensorflow as tf
from tensorflow import keras

# Membuat strategi untuk menggunakan semua GPU yang tersedia
strategy = tf.distribute.MirroredStrategy()

# Semua kode untuk membuat dan meng-compile model harus berada di dalam scope ini
with strategy.scope():
    # Model yang sama seperti biasanya
    model = keras.models.Sequential([
        keras.layers.Dense(30, activation="relu", input_shape=input_shape),
        keras.layers.Dense(1)
    ])
    optimizer = keras.optimizers.SGD(learning_rate=0.01)
    model.compile(loss="mse", optimizer=optimizer)

# Proses .fit() berjalan seperti biasa, TensorFlow akan menangani distribusinya
# model.fit(train_dataset, epochs=10, validation_data=valid_dataset)